# Seguradora de Automóveis Tera S.A.

Você acaba de ser contratado como cientista de dados para a Seguradora de Automóveis Tera S.A. e está encubido de auxiliar a equipe atuarial a determinar melhorias na tabela atualmente praticada nos seguros de automóveis nos Estados Unidos. Diversos estudos relativos a demografia foram feitos e incorporados na tabela atual, restando apenas analisar região a região quais as possíveis economias.

Para isso, <b>você deve utilizar os dados de acidentes rodoviários dos Estados Unidos do período de fevereiro de 2016 a dezembro de 2019 para dar insumos proveitosos para a equipe atuarial</b>. Além disso, também é esperado que você apresente suas descobertas para a diretoria de vendas, pois as alterações na tabela resultarão em uma mudança no plano de negócios atual. Portanto, <b>determine a melhor forma de apresentar graficamente o que realmente irá convencer a diretoria de que as mudanças na tabela atuarial irão resultar em ganhos financeiros para a empresa como um todo.</b>

# Tratativa da base

Antes de começar a ter qualquer insight, será necessário limpar a base que será trabalhada. Para isso, é necessário importar as bibliotecas que serão utilizadas e o arquivo do dataset.

In [2]:
# importando todas as bibliotecas importantes para cálculo e criação de visualização gráfica
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# o %matplotlib inline é um comando que faz com que os gráficos abram no mesmo notebook que estamos trabalhando
%matplotlib inline
import seaborn as sns
import plotly.express as px



# importando o arquivo em si, usando encoding utf-8 (como é um arquivo americano cobre todos os caracteres), em seguida vemos
# um .head() para ver se a exportação de fato foi correta
importa_df = pd.read_csv('US_Accidents_March23.csv', encoding='utf-8', sep= ',')
importa_df.head()



,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


Como o arquivo é muito grande, vamos reduzí-lo para 100.000 linhas

In [4]:
# utilizando as 100.000 primeiras linhas do arquivo (é possível utilizar as 100.000 últimas também)
acidentes  = importa_df.head(100000)
acidentes.head()



,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


Garanta que não existam linhas duplicadas no seu dataset, isso pode enviesar sua análise no final.

In [5]:
# verificando a quantidade de linhas e colunas antes de remover as duplicatas
acidentes.shape

(100000, 46)

In [6]:
# removendo as duplicatas e verificando quantas linhas e colunas restaram
# neste caso não houve nenhuma duplicata :)
acidentes = acidentes.drop_duplicates()
acidentes.shape



(100000, 46)

Vamos trocar a nomenclatura das colunas para deixar a interpretação mais fácil tanto para a equipe atuarial quanto para a equipe de vendas.

In [10]:
# primeiramente ver o nome de todas as colunas e as informações que elas trazem
acidentes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     100000 non-null  object 
 1   Source                 100000 non-null  object 
 2   Severity               100000 non-null  int64  
 3   Start_Time             100000 non-null  object 
 4   End_Time               100000 non-null  object 
 5   Start_Lat              100000 non-null  float64
 6   Start_Lng              100000 non-null  float64
 7   End_Lat                0 non-null       float64
 8   End_Lng                0 non-null       float64
 9   Distance(mi)           100000 non-null  float64
 10  Description            100000 non-null  object 
 11  Street                 100000 non-null  object 
 12  City                   99999 non-null   object 
 13  County                 100000 non-null  object 
 14  State                  100000 non-nul

In [11]:
#após trocar o nome de colunas confusas e dúbias, é importante usar o info novamente para ver se a aplicação foi bem sucedida



Tratar as células que contém data, para garantir que estão no formato de data legível pelo Python

In [12]:
# fórmulas para tratar as colulas Start_Time e End_Time
acidentes.Start_Time = pd.to_datetime(acidentes.Start_Time)
acidentes.End_Time = pd.to_datetime(acidentes.End_Time)





In [13]:
# validação se as variáveis estão em formato de data
acidentes.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   ID                     100000 non-null  object        
 1   Source                 100000 non-null  object        
 2   Severity               100000 non-null  int64         
 3   Start_Time             100000 non-null  datetime64[ns]
 4   End_Time               100000 non-null  datetime64[ns]
 5   Start_Lat              100000 non-null  float64       
 6   Start_Lng              100000 non-null  float64       
 7   End_Lat                0 non-null       float64       
 8   End_Lng                0 non-null       float64       
 9   Distance(mi)           100000 non-null  float64       
 10  Description            100000 non-null  object        
 11  Street                 100000 non-null  object        
 12  City                   99999 non-null   objec

Vamos verificar em quais colunas há valores nulos e se há a necessidade de preenchê-los

In [14]:
# soma todos os valores nulos por coluna
acidentes.isnull().sum()


ID                            0
Source                        0
Severity                      0
Start_Time                    0
End_Time                      0
Start_Lat                     0
Start_Lng                     0
End_Lat                  100000
End_Lng                  100000
Distance(mi)                  0
Description                   0
Street                        0
City                          1
County                        0
State                         0
Zipcode                       7
Country                       0
Timezone                      7
Airport_Code                  7
Weather_Timestamp          1054
Temperature(F)             1591
Wind_Chill(F)             95678
Humidity(%)                1856
Pressure(in)               1292
Visibility(mi)             1846
Wind_Direction             1064
Wind_Speed(mph)           23820
Precipitation(in)         92632
Weather_Condition          1604
Amenity                       0
Bump                          0
Crossing

Podemos ver que temos muitas variáveis que possuem valor nulo. Abaixo seguem listadas quais são elas e como elas devem ser tratadas quando nulas.<br>
<ul>
    <li>End Lat - preencher com "N/A"</li>
    <li>End Lng - preencher com "N/A"</li>
    <li>Street_Number - preencher com "N/A"</li>
    <li>City - preencher com "N/A"</li>
    <li>Sunrise_Sunset - preencher com "N/A"</li>
    <li>Civil_Twilight - preencher com "N/A"</li>
    <li>Nautical_Twilight - preencher com "N/A"</li>
    <li>Astronomical_Twilight - preencher com "N/A"</li>
    <li>Timezone - preencher com "N/A"</li>
    <li>Airport Code - preencher com "N/A"</li>
    <li>Weather Timestamp - preencher com "N/A"</li>
    <li>Wind_Direction - preencher com "N/A"</li>
    <li>Weather_Condition - preencher com "N/A"</li>
    <li>Traffic_Message_Channel - preencher com o valor 999 ou com o valor 0</li>
    <li>Zipcode - preencher com 99999</li>
    <li>Temperature (F) - preencher com 999</li>
    <li>Wind_Chill(F) - preencher com 999</li>
    <li>Humidity(%) - preencher com 0</li>
    <li>Pressure(in) - preencher com 0</li>
    <li>Visibility(mi) - preencher com 0</li>
    <li>Wind_Speed(mph) - preencher com 0</li>
    <li>Precipitation(in) - preencher com 0</li>
</ul>

In [20]:
# preenchendo End Lat, End Lng, Street_Number, City, Sunrise_Sunset, Civil_Twilight e Astronomical_Twilight com "N/A"
acidentes['End_Lat'] = acidentes['End_Lat'].fillna('N/A')
acidentes['End_Lng'] = acidentes['End_Lng'].fillna('N/A')
acidentes['Street'] = acidentes['Street'].fillna('N/A')
acidentes['City'] = acidentes['City'].fillna('N/A')
acidentes['Sunrise_Sunset'] = acidentes['Sunrise_Sunset'].fillna('N/A')
acidentes['Civil_Twilight'] = acidentes['Civil_Twilight'].fillna('N/A')
acidentes['Nautical_Twilight'] = acidentes['Nautical_Twilight'].fillna('N/A')
acidentes['Astronomical_Twilight'] = acidentes['Astronomical_Twilight'].fillna('N/A')
acidentes['Timezone'] = acidentes['Timezone'].fillna('N/A')
acidentes['Airport_Code'] = acidentes['Airport_Code'].fillna('N/A')
acidentes['Weather_Timestamp'] = acidentes['Weather_Timestamp'].fillna('N/A')
acidentes['Wind_Direction'] = acidentes['Wind_Direction'].fillna('N/A')
acidentes['Weather_Condition'] = acidentes['Weather_Condition'].fillna('N/A')



# preenchendo a coluna Traffic_Message_Channel



# preenchendo Zipcode com 99999
acidentes['Zipcode'] = acidentes['Zipcode'].fillna(99999)


# preenchendo Temperature(F) e Wind Chill(F) com 999
acidentes['Temperature(F)'] = acidentes['Temperature(F)'].fillna(999)
acidentes['Wind_Chill(F)'] = acidentes['Wind_Chill(F)'].fillna(999)


# preenchendo Humidity, Pressure, Visibility, Wind Speed e Precipitation com 0
acidentes['Humidity(%)'] = acidentes['Humidity(%)'].fillna(0)
acidentes['Pressure(in)'] = acidentes['Pressure(in)'].fillna(0)
acidentes['Visibility(mi)'] = acidentes['Visibility(mi)'].fillna(0)
acidentes['Wind_Speed(mph)'] = acidentes['Wind_Speed(mph)'].fillna(0) 
acidentes['Precipitation(in)'] = acidentes['Precipitation(in)'].fillna(0)










In [21]:
# validando se todos os campos estão preenchidos
acidentes.isnull().sum()


ID                       0
Source                   0
Severity                 0
Start_Time               0
End_Time                 0
Start_Lat                0
Start_Lng                0
End_Lat                  0
End_Lng                  0
Distance(mi)             0
Description              0
Street                   0
City                     0
County                   0
State                    0
Zipcode                  0
Country                  0
Timezone                 0
Airport_Code             0
Weather_Timestamp        0
Temperature(F)           0
Wind_Chill(F)            0
Humidity(%)              0
Pressure(in)             0
Visibility(mi)           0
Wind_Direction           0
Wind_Speed(mph)          0
Precipitation(in)        0
Weather_Condition        0
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
S

Agora que toda a base está tratada, vamos observar alguns dados gerais antes de começar nossas análises de fato? Vamos começar olhando dados estatísticos gerais da base.

In [22]:
# observar dados estatísticos da base
acidentes.describe()


,Severity,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,2.448120,35.630369,-119.322756,0.011345,81.475090,957.934336,58.684390,29.573924,9.166248,6.128045,0.001900
std,0.499931,2.101259,3.663525,0.262244,117.342926,193.260979,24.344801,3.394567,2.251865,5.081416,0.015565
min,1.000000,32.542587,-123.813927,0.000000,3.200000,-8.400000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,33.957775,-121.828468,0.000000,57.900000,999.000000,42.000000,29.890000,10.000000,3.500000,0.000000
50%,2.000000,34.168579,-118.384232,0.000000,66.200000,999.000000,61.000000,29.970000,10.000000,5.800000,0.000000
75%,3.000000,37.776682,-117.916023,0.010000,75.200000,999.000000,78.000000,30.060000,10.000000,9.200000,0.000000
max,4.000000,41.428753,-81.550728,51.130000,999.000000,999.000000,100.000000,33.040000,80.000000,241.700000,0.550000


# Análises auxiliares para equipe atuarial

Algumas sugestões de relações que podem impactar no estudo atuarial por região:<br>
<br>
<ul>
    <li>Severidade do acidente versus região</li>
    <li>Severidade do acidente versus região versus horário do dia (manhã/noite)</li>
    <li>Severidade do acidente versus temperatura versus precipitação</li>
    <li>Quantidade de acidentes por severidade por região</li>
</ul>

Análise de Severidade de Acidente por Região

In [30]:
# análise de severidade de acidente por região
acidente_regiao = acidentes.groupby(['Severity', 'City'])['Severity'].count().reset_index(name = 'Qtd')
acidente_regiao




,Severity,City,Qtd
0,1,Alpine,1
1,1,American Canyon,1
2,1,Bellflower,1
3,1,Cambridge,1
4,1,Canyon Country,1
...,...,...,...
1262,4,San Diego,2
1263,4,San Jose,2
1264,4,San Leandro,1
1265,4,Santa Ana,1


In [32]:
# análise gráfica interativa
fig = px.bar(acidente_regiao, x = 'City', y = 'Qtd', hover_data = ['Severity'], labels= {'City': 'Cidade', 'Qtd':  'Quantidade de Acidentes'}, color = 'Qtd')
fig.show()


Análise da Severidade do Acidente Versus Região Versus Horário do Dia

In [33]:
# análise de severidade do acidente versus região versus horário do dia (manhã/noite)
acidente_sunrise = acidentes.groupby(['Severity', 'City', 'Sunrise_Sunset'])['Severity'].count().reset_index(name='Qtd')
acidente_sunrise



,Severity,City,Sunrise_Sunset,Qtd
0,1,Alpine,Day,1
1,1,American Canyon,Day,1
2,1,Bellflower,Day,1
3,1,Cambridge,Day,1
4,1,Canyon Country,Night,1
...,...,...,...,...
2182,4,San Jose,Day,1
2183,4,San Jose,Night,1
2184,4,San Leandro,Day,1
2185,4,Santa Ana,Night,1


In [39]:
# gráfico scatterplot interativo
graph = px.scatter(acidente_sunrise, x = 'City', y = 'Sunrise_Sunset', color = 'Severity', size = 'Qtd', hover_data = ['Severity'])
graph.show()



Análise de Severidade do Acidente Versus Temperatura Versus Precipitação

In [17]:
# análise de severidade do acidente versus temperatura versus precipitação




In [18]:
#análise de lmplot



Análise de Quantidade de Acidentes por Severidade por Região

In [19]:
# análise de quantidade de acidentes por severidade por região





In [20]:
# gráfico de barras agrupadas


